In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments
from adapters import AutoAdapterModel, AdapterConfig, AdapterTrainer
from adapters.composition import Stack
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 1. Setup and Imports
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Load Model and Tokenizer
model_name = "bert-base-uncased"
model = AutoAdapterModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3. Load Dataset
dataset = load_dataset("glue", "sst2")

# 4. Tokenize Dataset
def tokenize_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 5. Add Adapters
adapter_config = AdapterConfig.load("pfeiffer", reduction_factor=16)
model.add_adapter("adapter1", config=adapter_config)
model.add_adapter("adapter2", config=adapter_config)
model.add_classification_head("sst2", num_labels=2)

# 6. Set Active Adapters and Train Adapter
model.set_active_adapters(Stack("adapter1", "adapter2"))
model.train_adapter(["adapter1", "adapter2"])

# 7. Prepare for Training
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=1e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# 8. Define Metrics Function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# 9. Initialize Trainer
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

# 10. Train the Model
trainer.train()

# 11. Evaluate the Model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

# 12. Inference
text = "This movie was fantastic!"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.softmax(outputs.logits, dim=-1)
    label = torch.argmax(predictions, dim=-1)

print(f"Input: {text}")
print(f"Prediction: {'Positive' if label.item() == 1 else 'Negative'}")
print(f"Confidence: {predictions[0][label.item()].item():.4f}")

# 13. Save the Model and Adapters
model.save_all_adapters("./adapters_save_directory")
model.save_head("./adapters_save_directory")

# 14. Load the Model and Adapters (for future use)
loaded_model = AutoAdapterModel.from_pretrained(model_name)
loaded_model.load_adapter("./adapters_save_directory/adapter1")
loaded_model.load_adapter("./adapters_save_directory/adapter2")
loaded_model.load_head("./adapters_save_directory/sst2")
loaded_model.set_active_adapters(Stack("adapter1", "adapter2"))
